<a href="https://colab.research.google.com/github/Syed-Raza-Ali/MindCare_Assistant/blob/main/MindCare_Assistant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##**"MindCare Assistant: Your Mental Health Guide"**

MindCare Assistant is an AI-powered chatbot designed to support your mental well-being. Specializing in mental health, it provides reliable, compassionate, and educational guidance on a variety of topics such as mental health conditions, symptoms, causes, treatments, and support resources. Whether you're seeking to learn about conditions like depression or strategies to improve your emotional health, MindCare Assistant is here to help.


- **NOTE:** This chatbot is solely for health-related assistance and does not replace professional medical advice.:

In [ ]:
# Install all necessary packages
!pip install -q langchain-core langchain-google-genai langgraph python-dotenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.5 MB/s eta 0:00:00


In [ ]:
import os
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langchain_google_genai import ChatGoogleGenerativeAI
from langgraph.graph import MessagesState, StateGraph, START, END
from langgraph.graph.state import CompiledStateGraph
from langgraph.checkpoint.memory import MemorySaver
from google.colab import userdata


GOOGLE_API_KEY = userdata.get("GEMINI_API_KEY")
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY


# Disable LangSmith Tracing programmatically
langchain_api_key = userdata.get("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_API_KEY"] = langchain_api_key
os.environ["LANGCHAIN_TRACING_V2"] = "false"
os.environ["LANGCHAIN_PROJECT"] = "Health-Agent"

# Initialize the Gemini LLM model with Google API Key
model = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    api_key = GOOGLE_API_KEY
)


# LangGraph nodes
def call_model(state: MessagesState):
    """Core conversational logic."""
    system_message = "You are an assistant specializing in mental health care. Your purpose is to provide accurate, supportive, and clear information about mental health conditions and related topics. if user not provide you any specific disease ask users for specific details about their condition or concern to tailor your response effectively; offer guidance on symptoms, causes, types, treatments, and support strategies for mental health conditions; at the end your response you can give one line solution in your responce to enhace user experiance; include an alert note in final response, reminding users to consult a doctor or mental health professional for accurate diagnosis and treatment; and avoid answering questions unrelated to mental health care by politely directing users to seek other resources for unrelated queries."
    messages = [SystemMessage(content=system_message)] + state["messages"]
    response = model.invoke(messages)
    return {"messages": response}


# Define the workflow
workflow = StateGraph(MessagesState)
workflow.add_node("conversation", call_model)
workflow.add_edge(START, "conversation")
workflow.add_edge("conversation", END)


# Compile the graph with memory
memory = MemorySaver()
graph = workflow.compile(checkpointer=memory)


# Chatbot runtime logic
def chatbot():
    """Run the chatbot."""
    config = {"configurable": {"thread_id": "1"}}

    print("Hello dear, i am your MindCare Assistant, My expertise is about mental health")
    print("Note: I have been designed exclusively as a MindCare assistant.i will not able to response any other questions")

    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Goodbye! Thank you for your trust, I hope this information is helpful for you")
            break


        input_message = HumanMessage(content=user_input)

        try:
            output = graph.invoke({"messages": [input_message]}, config)
        except Exception as e:
            print(f"Error: {str(e)}")
            continue

        bot_response = output["messages"][-1].content
        print(f"Chatbot: {bot_response}")

        state = graph.get_state(config)



# Run the chatbot
chatbot()



Hello dear, i am your MindCare Assistant, My expertise is about mental health
Note: I have been designed exclusively as a MindCare assistant.i will not able to response any other questions
You: how can i reduce stress?
Chatbot: Stress is a common experience, and managing it effectively is crucial for overall well-being.  To best help you reduce your stress, I need a little more information.  Could you tell me more about the sources of your stress?  For example:

* **What situations or events are causing you stress?** (e.g., work deadlines, relationship problems, financial worries, major life changes)
* **What are your physical and emotional symptoms of stress?** (e.g., headaches, muscle tension, irritability, difficulty sleeping, anxiety)
* **What coping mechanisms have you tried already?** (e.g., exercise, meditation, talking to friends, etc.)  Knowing what's worked and what hasn't will help me suggest more tailored strategies.


Once I have a better understanding of your specific sit